# Running the SUMMA setup
To begin, we have to regionalize the paths in the configuration files that SUMMA will use.
This is accomplished by running a shell command. This is done by starting a line with the `!` operator.
We simply run a script to complete the installation.
Then, we can import some basic libraries along with `pysumma`.
The `%pylab inline` magic command simply imports some standard scientific packages such as `numpy` and `matplotlib`.

<br>

### You will need to edit these paths to be your folders

In [ ]:
top = '/glade/work/ashleyvb'
folder = top+'/CAMELs'
folders = folder+'/summa_camels'
! cd /glade/work/ashleyvb/CAMELs/summa_camels; ./install_local_setup.sh

First we check that we loaded the correct environment.

In [ ]:
conda list pysumma

<br>
Then we load the imports.

In [ ]:
%pylab inline
import pysumma as ps
import xarray as xr
import pandas as pd
from dask_jobqueue import PBSCluster
from dask.distributed import Client

In [ ]:
NCORES=48
cluster = PBSCluster(n_workers = NCORES,
                     cores=NCORES,
                     processes=NCORES, 
                     memory="24GB",
                     project='UWAS0091',
                     queue='regular',
                     walltime='06:00:00')
client = Client(cluster)

### Check that have workers, do not run the rest of the cells until the workers show up. 

In [ ]:
print(client)
!qstat

### <br>

# Interacting with SUMMA via the `Distributed` object

We are running a `Distributed` object, which has multiple `Simulation` objects inside, each corresponding to some spatial chunk. 
We need to do `rm -r /glade/work/ashleyvb/CAMELs/summa_camels/.pysumma/` to clear out the distributed folders every run so permissions do not get screwed up in the loops. 

In [ ]:
# for fewer basins, do not exceed number of basins in chunking
CHUNK = 8 #for all 671 basins
# get number of HRUs
attrib = xr.open_dataset(folders+'/settings.v1/attributes.nc')
the_hru = np.array(attrib['hruId'])
if len(the_hru) <8: CHUNK = len(the_hru)

<br>

## Instantiating a `Distributed` object

To set up a `Distributed` object you must supply several pieces of information. 
First, supply the SUMMA executable; this could be either the compiled executable on your local machine, or a docker image. 
The second piece of information is the path to the file manager, which we just created through the install script. 

In [ ]:
executable = top+'/summa/bin/summa.exe'

In [ ]:
file_manager = folders+'/file_manager_truth.txt'
camels = ps.Distributed(executable, file_manager, num_workers=NCORES, chunk_size=CHUNK, client=client)

<br>

## Manipulating the configuration of the distributed object

Currrently, none of the attributes that can be changed in a SUMMA `Simulation` object cannot be altered in a `Distributed` object and the only one that can be viewed is the file manager. In the notebook that made the forcing files, we wrote file managers. 
To see a file manager, simply `print` it.

In [ ]:
print(camels.manager) #possible days 1980-01-01 to 2018-12-31, we are running 1986-10-01 01:00 to 1991-10-02 0:00

<br>

## Running pySUMMA for all Forcing Files

We now run all 671 basins of pySUMMA for each set of forcing files. You can check how long it has been running by using the command `qstat -u <username>` in a terminal. Each run takes about 9 minutes. First, we start with the original NLDAS files, or the "truth run".

In [ ]:
%%time
camels.run('local')
#all_status = [(n, s.status) for n, s in camels.simulations.items()] #if want to look at status if has errors
all_ds = [s.output.load() for n, s in camels.simulations.items()] #load it into memory so faster

<br>
We could just write it as several files instead of merging. However, if we want to merge, we can do the following.
First, detect automatically which vars have hru vs gru dimensions (depending on what we use for output, we may not have any gru):

In [ ]:
hru_vars = [] # variables that have hru dimension
gru_vars = [] # variables that have gru dimension
for ds in all_ds:
    for name, var in ds.variables.items():
        if 'hru' in var.dims:
            hru_vars.append(name)
        elif 'gru' in var.dims:
            gru_vars.append(name)

<br>
Filter variables for merge, this takes seconds since we are running a limiited output, but if you add more to the output it will take longer.

In [ ]:
%%time
hru_ds = [ds[hru_vars] for ds in all_ds]
gru_ds = [ds[gru_vars] for ds in all_ds]
hru_merged = xr.concat(hru_ds, dim='hru')
gru_merged = xr.concat(gru_ds, dim='gru')

In [ ]:
print(hru_merged)

In [ ]:
%%time
hru_merged.to_netcdf(folders+'/output/merged_day/NLDAStruth_hru.nc')
gru_merged.to_netcdf(folders+'/output/merged_day/NLDAStruth_gru.nc')
del camels
del all_ds 
del hru_merged
del gru_merged

<br>
Here are the other runs, now as a loop. The processes are the same, but for clarity we will divide it into 2 loops, one for the constant forcings and one for the MetSim forcings. This will take about an hour for each loop using all 671 basins. We delete stuff after every run to reduce memory needs.

In [ ]:
%%time
# Constant
constant_vars= ['airpres','airtemp','LWRadAtm','pptrate','spechum','SWRadAtm','windspd','all']
for v in constant_vars:
    ! rm -rf /glade/work/ashleyvb/CAMELs/summa_camels/.pysumma 
    file_manager = folders+'/file_manager_constant_' + v +'.txt'
    camels = ps.Distributed(executable, file_manager, num_workers=NCORES, chunk_size=CHUNK, client=client)   
    camels.run('local')
    #all_status = [(n, s.status) for n, s in camels.simulations.items()] #if want to look at status if has errors
    all_ds = [s.output.load() for n, s in camels.simulations.items()] #load it into memory so faster    
    hru_vars = [] # variables that have hru dimension
    gru_vars = [] # variables that have gru dimension
    for ds in all_ds:
        for name, var in ds.variables.items():
            if 'hru' in var.dims:
                hru_vars.append(name)
            elif 'gru' in var.dims:
                gru_vars.append(name)
    hru_ds = [ds[hru_vars] for ds in all_ds]
    gru_ds = [ds[gru_vars] for ds in all_ds]
    hru_merged = xr.concat(hru_ds, dim='hru')
    gru_merged = xr.concat(gru_ds, dim='gru')
    hru_merged.to_netcdf(folders+'/output/merged_day/NLDASconstant_' + v +'_hru.nc')
    gru_merged.to_netcdf(folders+'/output/merged_day/NLDASconstant_' + v +'_gru.nc')
    del camels
    del all_ds 
    del hru_merged
    del gru_merged
    print(v)

In [ ]:
%%time
# Metsim
metsim_vars= ['airpres','airtemp','LWRadAtm','pptrate','spechum','SWRadAtm','windspd','all']
for v in metsim_vars:
    ! rm -rf /glade/work/ashleyvb/CAMELs/summa_camels/.pysumma 
    file_manager = folders+'/file_manager_metsim_' + v +'.txt'
    camels = ps.Distributed(executable, file_manager, num_workers=NCORES, chunk_size=CHUNK, client=client)   
    camels.run('local')
    #all_status = [(n, s.status) for n, s in camels.simulations.items()] #if want to look at status if has errors
    all_ds = [s.output.load() for n, s in camels.simulations.items()] #load it into memory so faster    
    hru_vars = [] # variables that have hru dimension
    gru_vars = [] # variables that have gru dimension
    for ds in all_ds:
        for name, var in ds.variables.items():
            if 'hru' in var.dims:
                hru_vars.append(name)
            elif 'gru' in var.dims:
                gru_vars.append(name)
    hru_ds = [ds[hru_vars] for ds in all_ds]
    gru_ds = [ds[gru_vars] for ds in all_ds]
    hru_merged = xr.concat(hru_ds, dim='hru')
    gru_merged = xr.concat(gru_ds, dim='gru')
    hru_merged.to_netcdf(folders+'/output/merged_day/NLDASmetsim_' + v +'_hru.nc')
    gru_merged.to_netcdf(folders+'/output/merged_day/NLDASmetsim_' + v +'_gru.nc')
    del camels
    del all_ds 
    del hru_merged
    del gru_merged
    print(v)